In [ ]:
import nnabla as nn

import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
from nnabla.monitor import tile_images

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

import copy
import scipy.ndimage.interpolation

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
lr = 1e-2

In [ ]:
def G_shared(z, hidden=[128]):
    hs = []
    with nn.parameter_scope("G_shared"):  # Parameter scope can be nested
        h = z
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("last"):
            H = F.relu(PF.affine(h, X_dim))
    return H

In [ ]:
def G1(z, hidden=[128]):
    h = G_shared(z)
    hs = []
    with nn.parameter_scope("G1"):  # Parameter scope can be nested
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("sigmoid_layer"):
            X = F.sigmoid(PF.affine(h, X_dim))
    return X

In [ ]:
def G2(z, hidden=[128]):
    h = G_shared(z)
    hs = []
    with nn.parameter_scope("G2"):  # Parameter scope can be nested
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("sigmoid_layer"):
            X = F.sigmoid(PF.affine(h, X_dim))
    return X

In [ ]:
def D_shared(h, hidden=[128]):
    hs = []
    with nn.parameter_scope("D_shared"):
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("classifier"):
            y = F.sigmoid(PF.affine(h, 1))
    return y

In [ ]:
def D1(X, hidden=[128]):
    hs = []
    with nn.parameter_scope("D1"):
        h = X
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("last_layer"):
            h = F.relu(PF.affine(h, h_dim))
    y = D_shared(h)
    return y

In [ ]:
def D2(X, hidden=[128]):
    hs = []
    with nn.parameter_scope("D2"):
        h = X
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("last_layer"):
            h = F.relu(PF.affine(h, h_dim))
    y = D_shared(h)
    return y

In [ ]:
G_solver = S.Adam(lr)
with nn.parameter_scope("G_shared"), nn.parameter_scope("G1"), nn.parameter_scope("G2"):
    G_solver.set_parameters(nn.get_parameters())
    
D_solver = S.Adam(lr)
with nn.parameter_scope("D_shared"), nn.parameter_scope("D1"), nn.parameter_scope("D2"):
    D_solver.set_parameters(nn.get_parameters())

In [ ]:
def reset_grad():
    G_solver.zero_grad()
    D_solver.zero_grad()

In [ ]:
def sample_X(X, size):
    start_idx = np.random.randint(0, X.shape[0]-size)
    return nn.Variable.from_numpy_array(X[start_idx:start_idx+size])

In [ ]:
def show16(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    plt.show()

In [ ]:
X_train = mnist.train.images
half = int(X_train.shape[0] / 2)

# Real image
X_train1 = X_train[:half]
# Rotated image
X_train2 = X_train[half:].reshape(-1, 28, 28)
X_train2 = scipy.ndimage.interpolation.rotate(X_train2, 90, axes=(1, 2))
X_train2 = X_train2.reshape(-1, 28*28)

# Cleanup
del X_train

In [ ]:
for it in range(1000000):
    # Discriminator
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    X1 = sample_X(X_train1, mb_size)
    X2 = sample_X(X_train2, mb_size)
    
    G1_sample = G1(z)
    D1_real = D1(X1)
    D1_fake = D1(G1_sample)
    
    G2_sample = G2(z)
    D2_real = D2(X2)
    D2_fake = D2(G1_sample)

    D1_loss = F.mean(-F.log(D1_real + 1e-8) - F.log(1. - D1_fake + 1e-8))
    D2_loss = F.mean(-F.log(D2_real + 1e-8) - F.log(1. - D2_fake + 1e-8))
    D_loss = D1_loss + D2_loss
    
    D_loss.forward()
    D_loss.backward()
    D_solver.update()
    
    reset_grad()

    # Generator Update
    G1_sample = G1(z)
    D1_fake = D1(G1_sample)

    G2_sample = G2(z)
    D2_fake = D2(G2_sample)
    
    G1_loss = F.mean(-F.log(D1_fake + 1e-8))
    G2_loss = F.mean(-F.log(D2_fake + 1e-8))
    G_loss = G1_loss + G2_loss

    G_loss.forward()
    G_loss.backward()
    G_solver.update()

    reset_grad()

    
    # Generate and Show Samples 
    if it % 1000 == 0:
        idx = np.random.randint(0, 10)
        print('Step: {}, D1_loss: {}, G1_loss: {}, D2_loss: {}, G2_loss: {}'.format(it, D1_loss.d, G1_loss.d, D2_loss.d, G2_loss.d))
#         print('Step: {}, D_loss: {}, G_loss: {}'.format(it, D_loss.d, G_loss.d))
        
        z = nn.Variable.from_numpy_array(np.random.randn(8, Z_dim))
        samples1 = G1(z).d
        samples2 = G2(z).d
        samples = np.vstack([samples1, samples2])

        show16(samples)